# Need for speed

**Table of contents**<a id='toc0_'></a>    
- 1. [Timing and precomputations](#toc1_)    
- 2. [Line profilling](#toc2_)    
- 3. [Optimizing Numpy](#toc3_)    
  - 3.1. [Tip 1: Always use vectorized operations when available](#toc3_1_)    
  - 3.2. [Tip 2: Operations are faster on rows than on columns](#toc3_2_)    
  - 3.3. [Tip 3: Also use vectorized operations when it is a bit cumbersome](#toc3_3_)    
- 4. [Numba](#toc4_)    
  - 4.1. [Further speed-up](#toc4_1_)    
  - 4.2. [Calling an optimizer](#toc4_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import numba as nb
import quantecon as qe
from scipy import optimize

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# magics
#  conda install line_profiler

%load_ext line_profiler

## 1. <a id='toc1_'></a>[Timing and precomputations](#toc0_)

Consider the following function doing some simple algebraic operations:

In [2]:
def myfun(x,i):
    y = 0
    for j in range(100):
        y += x**j
    return y + i

And another function calling the former function in a loop:

In [3]:
def myfun_loop(n):
    mysum = 0
    for i in range(n):
        mysum += myfun(5,i)
    return mysum

**How long does it take to run ``myfun_loop``:**

**A.** Manual timing

In [4]:
t0 = time.perf_counter()
mysum = myfun_loop(1000)
t1 = time.perf_counter()    
print(f'{t1-t0:.8} seconds')

0.0295001 seconds


**B.** Use the ``%time`` magic (work on a single line)

In [5]:
%time mysum = myfun_loop(1000)
%time mysum = myfun_loop(1000)

CPU times: total: 31.2 ms
Wall time: 29.9 ms
CPU times: total: 31.2 ms
Wall time: 31.7 ms


**ms** $\equiv$ milliseconds, $10^{-3}$ of a second.<br>
**$\mu$ s** $\equiv$ mikroseconds, $10^{-6}$ of a second.<br>
**ns** $\equiv$ nanoseconds, $10^{-9}$ of a second.

**C.** Use the ``%timeit`` magic to also see variability (work on single line)

In [6]:
%timeit myfun_loop(1000)
%timeit -r 5 -n 20 myfun_loop(1000)

49.8 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.3 ms ± 1.22 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


``%timeit`` report the best of ``r`` runs each calling the code ``n`` times in a loop

**Question:** How can we speed up the computation using **precomputation**?

In [7]:
# write your code
    
# remember
def myfun_loop(n):
    mysum = 0
    for i in range(n):
        mysum += myfun(5,i)
    return mysum

def myfun(x,i):
    y = 0
    for j in range(100):
        y += x**j
    return y + i

**Answer:**

In [8]:
def myfun_loop_fast(n):
    myfunx = myfun(5,0) # precomputation
    mysum = 0
    for i in range(n):
        mysum += myfunx + i
    return mysum

In [9]:
t0 = time.perf_counter()
mysum_fast = myfun_loop_fast(1000)
t1 = time.perf_counter()    
print(f'{t1-t0:.8f} seconds')

0.00021370 seconds


Too fast to be measured with ``time.perf_counter()``. The ``%timeit`` magic still works:

In [10]:
%timeit myfun_loop(1000)
%timeit myfun_loop_fast(1000)

31.3 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
137 µs ± 8.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


$\rightarrow$ **orders of magnitude faster!**

Check the **results are the same**:

In [11]:
assert mysum == mysum_fast

## 2. <a id='toc2_'></a>[Line profilling](#toc0_)

**Premature optimization is the root of all evil!**

**Important:** Before deciding whether to do a precomputation (which often makes the code harder to read) we should investigate, whether it alleviates a bottleneck.

* **A.** Insert multiple ``time.perf_counter()`` to time different parts of the code.
* **B.** Use the ``line_profiler`` with syntax (also works with methods for classes)

  ``%lprun -f FUNCTION_TO_PROFILE -f FUNCTION_TO_PROFILE FUNCTION_TO_RUN``

**Baseline method:**

In [12]:
%lprun -f myfun -f myfun_loop myfun_loop(1000)

Timer unit: 1e-07 s

Total time: 0.168759 s
File: C:\Users\gmf123\AppData\Local\Temp\ipykernel_25728\2824044186.py
Function: myfun_loop at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def myfun_loop(n):
     5         1          5.0      5.0      0.0      mysum = 0
     6      1001       4056.0      4.1      0.2      for i in range(n):
     7      1000    1683530.0   1683.5     99.8          mysum += myfun(5,i)
     8         1          3.0      3.0      0.0      return mysum

Total time: 0.110394 s
File: C:\Users\gmf123\AppData\Local\Temp\ipykernel_25728\2824044186.py
Function: myfun at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def myfun(x,i):
    11      1000       3465.0      3.5      0.3      y = 0
    12    101000     349560.0      3.5     31.7      for j in range(100):
    13    100000     746446.0      7.5     67.6          y +

**Observation:** Most of the time is spend in ``myfun()``, more specifically the computation of the power in line 4. The precomputation solves this problem.

**Compare with the fast method:**

In [13]:
%lprun -f myfun_loop_fast myfun_loop_fast(1000)

Timer unit: 1e-07 s

Total time: 0.0011151 s
File: C:\Users\gmf123\AppData\Local\Temp\ipykernel_25728\3913433440.py
Function: myfun_loop_fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def myfun_loop_fast(n):
     2         1       1355.0   1355.0     12.2      myfunx = myfun(5,0) # precomputation
     3         1          6.0      6.0      0.1      mysum = 0
     4      1001       3996.0      4.0     35.8      for i in range(n):
     5      1000       5789.0      5.8     51.9          mysum += myfunx + i
     6         1          5.0      5.0      0.0      return mysum

## 3. <a id='toc3_'></a>[Optimizing Numpy](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Tip 1: Always use vectorized operations when available](#toc0_)

**Simple comparison:**

In [14]:
x = np.random.uniform(size=500000)

def python_add(x):
    y = []
    for xi in x:
        y.append(xi+1)
    return y

def numpy_add(x):
    y = np.empty(x.size)
    for i in range(x.size):
        y[i] = x[i]+1
    return y

def numpy_add_vec(x):
    return x+1

assert np.allclose(python_add(x),numpy_add(x))
assert np.allclose(python_add(x),numpy_add_vec(x))

%timeit python_add(x)
%timeit numpy_add(x)
%timeit numpy_add_vec(x)

82.7 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
125 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.5 ms ± 88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Even **stronger** when the **computation is more complicated:**

In [15]:
def python_exp(x):
    y = []
    for xi in x:
        y.append(np.exp(xi))
    return y

def numpy_exp(x):
    y = np.empty(x.size)
    for i in range(x.size):
        y[i] = np.exp(x[i])
    return y

def numpy_exp_vec(x):
    return np.exp(x)

assert np.allclose(python_exp(x),numpy_exp(x))
assert np.allclose(python_exp(x),numpy_exp_vec(x))

%timeit python_exp(x)
%timeit numpy_exp(x)
%timeit numpy_exp_vec(x)

422 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
484 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.37 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Also works for a **conditional sum**:

In [16]:
def python_exp_cond(x):
    return [np.exp(xi) for xi in x if xi < 0.5]

def numpy_exp_vec_cond(x):
    y = np.exp(x[x < 0.5])
    return y

def numpy_exp_vec_cond_alt(x):
    y = np.exp(x)[x < 0.5]
    return y

assert np.allclose(python_exp_cond(x),numpy_exp_vec_cond(x))
assert np.allclose(python_exp_cond(x),numpy_exp_vec_cond_alt(x))

%timeit python_exp_cond(x)
%timeit numpy_exp_vec_cond(x)
%timeit numpy_exp_vec_cond_alt(x)

253 ms ± 7.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.86 ms ± 90.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.65 ms ± 717 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Question:** Why do you think the speed-up is less pronounced in this case?

### 3.2. <a id='toc3_2_'></a>[Tip 2: Operations are faster on rows than on columns](#toc0_)

Generally, operate on the **outermost index**.

In [17]:
n = 1000
x = np.random.uniform(size=(n,n))

def add_rowsums(x):
    mysum = 0
    for i in range(x.shape[0]):
        mysum += np.sum(np.exp(x[i,:]))
    return mysum
            
def add_colsums(x):
    mysum = 0
    for j in range(x.shape[1]):
        mysum += np.sum(np.exp(x[:,j]))
    return mysum

assert np.allclose(add_rowsums(x),add_colsums(x))
            
%timeit add_rowsums(x)
%timeit add_colsums(x)

15.2 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
31.6 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<img src="https://github.com/NumEconCopenhagen/lectures-2019/raw/master/11/numpy_memory_layout.png" alt="amdahls_law" width=60% />

The **memory structure can be changed manually** so that working on columns (innermost index) is better than working on rows (outermost index):

In [18]:
y = np.array(x,order='F') # the default is order='C'
%timeit add_rowsums(y)
%timeit add_colsums(y)

34.4 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.1 ms ± 369 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3.3. <a id='toc3_3_'></a>[Tip 3: Also use vectorized operations when it is a bit cumbersome](#toc0_)

Consider the task of calculating the following **expected value**:

$$
\begin{aligned}
W(a)&=\mathbb{E}\left[\sqrt{\frac{a}{\psi}+\xi}\right]\\
\psi,\xi&\in \begin{cases}
0.25 & \text{with prob. }0.25\\
0.5 & \text{with prob. }0.25\\
1.5 & \text{with prob. }0.25\\
1.75 & \text{with prob. }0.25
\end{cases}\end{aligned}
$$

for a vector of $a$-values.

**Setup:**

In [19]:
N = 5000
a_vec = np.linspace(0,10,N)

xi_vec = np.array([0.25,0.5,1.5,1.75])
psi_vec = np.array([0.25,0.5,1.5,1.75])

xi_w_vec = np.ones(4)/4
psi_w_vec = np.ones(4)/4

**Loop based solution:**

In [20]:
def loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec):
    
    w_vec = np.zeros(a_vec.size)
    for i,a in enumerate(a_vec):        
        for xi,xi_w in zip(xi_vec,xi_w_vec):
            for psi,psi_w in zip(psi_vec,psi_w_vec):
                m_plus = a/psi + xi
                v_plus = np.sqrt(m_plus)
                w_vec[i] += xi_w*psi_w*v_plus
    
    return w_vec
        
loop_result = loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec)  
%timeit loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec)      

136 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Vectorized solution:**

In [21]:
def vec(a,xi,psi,xi_w,psi_w):   
    m_plus_vec = a[:,np.newaxis,np.newaxis]/psi[np.newaxis,:,np.newaxis] + xi[np.newaxis,np.newaxis,:]
    v_plus_vec = np.sqrt(m_plus_vec)
    w_mat = xi_w[np.newaxis,np.newaxis,:]*psi_w[np.newaxis,:,np.newaxis]*v_plus_vec
    w_vec = np.sum(w_mat,axis=(1,2))
    return w_vec

vec_result = vec(a_vec,psi_vec,xi_vec,xi_w_vec,psi_w_vec)
assert np.allclose(loop_result,vec_result)
%timeit vec(a_vec,psi_vec,xi_vec,xi_w_vec,psi_w_vec)

941 µs ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


**Conclusion:** Much much faster.

## 4. <a id='toc4_'></a>[Numba](#toc0_)

Writing **vectorized code can be cumbersome**, and in some cases it is impossible. Instead we can use the **numba** module. 

Adding the decorator `nb.njit` on top of a function tells numba to compile this function **to machine code just-in-time**.

This takes some time when the function is called the first time, but subsequent calls are then a lot faster. 

*The input types can, however, not change between calls because numba infer them on the first call.*

In [22]:
def myfun_numpy_vec(x1,x2):

    y = np.empty((1,x1.size))
    I = x1 < 0.5

    y[I] = np.sum(np.exp(x2*x1[I]),axis=0)
    y[~I] = np.sum(np.log(x2*x1[~I]),axis=0)
    
    return y

# setup
x1 = np.random.uniform(size=10**6)
x2 = np.random.uniform(size=np.int64(100)) # adjust the size of the problem
x1_np = x1.reshape((1,x1.size))
x2_np = x2.reshape((x2.size,1))

# timing
%timeit myfun_numpy_vec(x1_np,x2_np)

1.29 s ± 47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Numba:** The first call is slower, but the result is the same, and the subsequent calls are faster:

In [23]:
@nb.njit
def myfun_numba(x1,x2):

    y = np.empty(x1.size)

    for i in range(x1.size):
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))
            
    return y

# call to just-in-time compile
%time myfun_numba(x1,x2)

# actual measurement
%timeit myfun_numba(x1,x2)

assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numba(x1,x2))

CPU times: total: 1.55 s
Wall time: 1.65 s
997 ms ± 71.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


You can also call the Python-version:

In [24]:
%timeit myfun_numba.py_func(x1,x2)

7.9 s ± 91.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Caveats:** Only a limited number of Python and Numpy features are supported inside just-in-time compiled functions.

- [Supported Python features](https://numba.pydata.org/numba-doc/dev/reference/pysupported.html)
- [Supported Numpy features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html)


### 4.1. <a id='toc4_1_'></a>[Further speed-up](#toc0_)

**Further speed up:** Use

1. parallelization (with ``prange``), and 
1. faster but less precise math (with ``fastmath``) (not always a speed-up)

In [25]:
@nb.njit(parallel=True)
def myfun_numba_par(x1,x2):

    y = np.empty(x1.size)
    
    for i in nb.prange(x1.size): # in parallel across threads
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))
            
    return y

assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numba_par(x1,x2))
%time myfun_numba_par(x1,x2)
%timeit myfun_numba_par(x1,x2)

CPU times: total: 1.5 s
Wall time: 200 ms
206 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
@nb.njit(parallel=True,fastmath=True)
def myfun_numba_par_fast(x1,x2):

    y = np.empty(x1.size)

    for i in nb.prange(x1.size): # in parallel across threads
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))
            
    return y

assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numba_par_fast(x1,x2))
%time myfun_numba_par_fast(x1,x2)
%timeit myfun_numba_par_fast(x1,x2)

CPU times: total: 1.44 s
Wall time: 198 ms
198 ms ± 7.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 4.2. <a id='toc4_2_'></a>[Calling an optimizer](#toc0_)

Using solver from `QuantEcon` (see [documentation](https://quanteconpy.readthedocs.io/en/latest/index.html)).

In [27]:
import quantecon as qe

In [28]:
n = 4000
rng = np.random.default_rng(1234)
alphas = rng.uniform(size=n)
betas = rng.uniform(size=n)
gammas = rng.uniform(size=n)

In [29]:
@nb.njit
def solver_nb(alpha,beta,gamma):

    def obj(x,alpha,beta,gamma):
        return (x[0]-alpha)**2 + (x[1]-beta)**2 + (x[2]-gamma)**2

    res = qe.optimize.nelder_mead(obj,np.array([0.0,0.0,0.0]),args=(alpha,beta,gamma))

    return res.x

**Serial version:**

In [30]:
@nb.njit
def serial_solver_nb(alphas,betas,gammas):

    n = alphas.size
    xopts = np.zeros((n,3))

    for i in range(n):
        xopts[i,:] = solver_nb(alphas[i],betas[i],gammas[i])

%time serial_solver_nb(alphas,betas,gammas)
%time serial_solver_nb(alphas,betas,gammas)

CPU times: total: 15.8 s
Wall time: 15.9 s
CPU times: total: 3.52 s
Wall time: 3.54 s


**Parallel version:**

In [31]:
@nb.njit(parallel=True)
def parallel_solver_nb(alphas,betas,gammas):

    n = alphas.size
    xopts = np.zeros((n,3))

    for i in nb.prange(n):
        xopts[i,:] = solver_nb(alphas[i],betas[i],gammas[i])

%time parallel_solver_nb(alphas,betas,gammas)
%time parallel_solver_nb(alphas,betas,gammas)

CPU times: total: 16.5 s
Wall time: 10.4 s
CPU times: total: 7.02 s
Wall time: 955 ms


**Alternative parallization:** Look at the `joblib` package